In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('./data/difford_1_5000.csv')

In [45]:
words_to_remove = ['Year Old','yo','Top up with','scoop', 'cube', 'fresh','shot','drop', 'spoon','dash', 'pinch']

alcohol = ['vodka', 
           'gin',
           'whiskey',
           'rum',
           'brandy liqueur',
           'Gentian liqueur',
           'soda water', 
           'amaretto', 
           'Calvados', 
           'bitters',
           'Chartreuse liqueur',
           'Coffee Liqueur',
           'bitter liqueur',
           'elderflower liqueur',
           'Maraschino liqueur',
           'absinthe',
           'tequila',
           'Orange Bitters',
           'vermouth',
           'Cognac',
           'Aromatic Bitters',
           'Hazelnut liqueur',
           'Vanilia vodka',
           'Chocolate bitters',
           'sherry',
           'sparkling wine',
           'Sauvignon blanc wine',
           'wine'
]


In [46]:
def extract_words(ingredient_list):
    cleaned_list = []
    for i,ingredient in enumerate(eval(ingredient_list)):
        ingredient = "".join(re.split("\(|\)|\[|\]", ingredient)[::2])
        clean_ingredient = ' '.join([word for word in re.findall("[a-zA-Z'é]+",ingredient) if word not in words_to_remove])

        cleaned_list.append(clean_ingredient)
    cleaned_string = '|'.join(clean_ingredient)
    return cleaned_list

    
def clean_serving(words):
    words_list = words.split('\n')
    return words_list[0]

def extract_alcohol(ingredients):
    matches = []
    alcohol_ingredients = []
    for ingredient in ingredients: 
        for i,item in enumerate(alcohol):
            index = ingredient.lower().find(item.lower())
            if index != -1:
                matches.append((i,len(item)))
        biggest_index = 0
        for i,tup in enumerate(matches):
            if tup[0] > biggest_index:
                biggest_index = i
        try:
            #print(word[word_i],' contains ' ,alcohol[matches[biggest_index][1]])
            alcohol_ingredients.append(alcohol[matches[biggest_index][0]])
        except:
            pass
            #print(ingredients[word_i],' contains no alcohol' )

        matches = []
    return "|".join(alcohol_ingredients)

def extract_non_alcohol(ingredients): #NYI
    matches = []
    alcohol_ingredients = []
    for ingredient in ingredients: 
        for i,item in enumerate(alcohol):
            index = ingredient.lower().find(item.lower())
            if index == -1:
                alcohol_ingredients.append(ingredient)

        matches = []
    return "|".join(alcohol_ingredients)


In [51]:
df['ingredients_clean'] = df['ingredients'].apply(extract_words)
df['serving_suggestion_clean'] = df['serving_suggestion'].apply(clean_serving)
df['alcohol'] = df['ingredients_clean'].apply(extract_alcohol)

In [52]:
df

,name,garnish,serving_suggestion,ingredients,steps,review_count,url,website,image_url,ingredients_clean,serving_suggestion_clean,alcohol
0,Abacaxi Ricaço,Cut a straw sized hole in the top of the pinea...,Serve in a Pineapple shell (frozen) glass,"['1 whole Pineapple (fresh)', '3 shot Havana C...",Cut the top off a small pineapple and carefull...,4,https://www.diffordsguide.com/cocktails/recipe...,diffords,https://cdn.diffords.com/contrib/stock-images/...,"[whole Pineapple, Havana Club Year Old rum, Li...",Serve in a Pineapple shell (frozen) glass,rum
1,Abbey,Orange zest twist,Serve in a Coupe glass,"['1 1⁄2 shot Rutte Dry Gin', '3⁄4 shot Lillet ...",SHAKE all ingredients with ice and fine strain...,11,https://www.diffordsguide.com/cocktails/recipe...,diffords,https://cdn.diffords.com/contrib/stock-images/...,"[Rutte Dry Gin, Lillet Blanc, Orange juice, An...",Serve in a Coupe glass,gin|Aromatic Bitters
2,A.B.C. Cocktail,Lemon zest twist & Luxardo Maraschino cherry,Serve in a Nick & Nora glass,"['7 fresh Mint leaves', '1 shot Tawny port', '...",TEAR mint and place in shaker. Add other ingre...,5,https://www.diffordsguide.com/cocktails/recipe...,diffords,https://cdn.diffords.com/contrib/stock-images/...,"[Mint leaves, Tawny port, Rémy Martin Cognac, ...",Serve in a Nick & Nora glass,Cognac|Maraschino liqueur
3,Absinthe Cocktail,Mint leaf,Serve in a Coupe glass,"['1 shot La Fée Parisienne absinthe', '2 1⁄2 s...",SHAKE all ingredients with ice and fine strain...,5,https://www.diffordsguide.com/cocktails/recipe...,diffords,https://cdn.diffords.com/contrib/stock-images/...,"[La Fée Parisienne absinthe, Chilled water, Su...",Serve in a Coupe glass,absinthe
4,Absinthe Frappé,Mint sprigs bouquet,Serve in an Old-fashioned glass,"['1 shot La Fée Parisienne absinthe', '1⁄3 sho...",SHAKE all ingredients with ice and fine strain...,1,https://www.diffordsguide.com/cocktails/recipe...,diffords,https://cdn.diffords.com/contrib/stock-images/...,"[La Fée Parisienne absinthe, Anisette liqueur,...",Serve in an Old-fashioned glass,absinthe
...,...,...,...,...,...,...,...,...,...,...,...,...
4739,Greek Daiquiri,Orange zest twist,Serve in a Coupe glass\nPhotographed in a Retr...,"['1 2⁄3 shot O/Purist Tsipouro', '2⁄3 shot Lim...",SHAKE all ingredients with ice and fine strain...,0,https://www.diffordsguide.com/cocktails/recipe...,diffords,https://cdn.diffords.com/contrib/stock-images/...,"[O Purist Tsipouro, Lime juice, Lemon juice, S...",Serve in a Coupe glass,Orange Bitters
4740,Cazador,Lime zest twist or dehydrated lime wheel slice,Serve in a Coupe glass,"['2 shot Bacardi 8 year old rum', '2⁄3 shot Li...",SHAKE all ingredients with ice and fine strain...,5,https://www.diffordsguide.com/cocktails/recipe...,diffords,https://cdn.diffords.com/contrib/stock-images/...,"[Bacardi year old rum, Lime juice, Maple syrup...",Serve in a Coupe glass,rum|absinthe
4741,Spiced Alexander,Dust with freshly grated nutmeg,Serve in a Coupe glass\nPhotographed in an UB ...,['1 1⁄2 shot Black Tears cacao & coffee spiced...,SHAKE all ingredients with ice and fine strain...,1,https://www.diffordsguide.com/cocktails/recipe...,diffords,https://cdn.diffords.com/contrib/stock-images/...,"[Black Tears cacao coffee spiced rum, Dutch Ca...",Serve in a Coupe glass,rum
4742,Sweet Liberty's Sherry Cobbler,Fresh raspberries & mint sprig bouquet,Serve in a Wine glass,"['2 1⁄2 shot Fino sherry', '1 shot Pink grapef...",SHAKE all ingredients with ice and strain into...,13,https://www.diffordsguide.com/cocktails/recipe...,diffords,https://cdn.diffords.com/contrib/stock-images/...,"[Fino sherry, Pink grapefruit juice, St Germai...",Serve in a Wine glass,sherry|elderflower liqueur


In [53]:
df.to_csv('./data/difford_1_5000_processed.csv', index=False)
df.to_parquet('./data/difford_1_5000_processed.parquet')

In [41]:
ingredient_dict = {}
ingredient_series = df['ingredients_clean']
for ingredients in ingredient_series:
    for ingredient in ingredients:
        if ingredient in list(ingredient_dict.keys()):
            ingredient_dict[ingredient] += 1
        else:
            ingredient_dict[ingredient] = 1

In [85]:
pd.set_option('max_rows', 100)
pd.DataFrame([ingredient_dict]).transpose().sort_values(0).head(100)

,0
whole Pineapple,1
Zoco Pacharan Navarro,1
Orange sherbet,1
Peroni lager,1
dried Cinnamon stick,1
Pine liqueur,1
Absinthe vermouth,1
Izarra Green liqueur,1
Giffard Mirabelle de Lorraine,1
Lapsang Souchong tea liqueur,1


In [ ]:
df[['name', 'steps','ingredients_clean']].to_parquet('test.parquet')